In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

In [ ]:
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive

In [5]:
import keras
from keras.applications import VGG16
from keras.models import Sequential, load_model
from keras import models, optimizers
from keras.optimizers import SGD
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.preprocessing.image import load_img, ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint

Using TensorFlow backend.


In [6]:
image_size = 224

vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False

model = models.Sequential()
model.add(vgg_conv)

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

model.summary()





58892288/58889256 [==============================] - 6s 0us/step







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 3075      
Total params: 40,408,899
Trainable params: 32,773,635
Non-tra

In [7]:
batch_size = 32
train_dir = 'drive/Colab Notebooks/data/train'
test_dir = 'drive/Colab Notebooks/data/test'

train_datagen = ImageDataGenerator(
    rescale = 1.0 / 255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True
)

validation_datagen = ImageDataGenerator(rescale = 1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (image_size, image_size),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = True
)

validation_generator = validation_datagen.flow_from_directory(
    test_dir,
    target_size = (image_size, image_size),
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = True
)

model.compile(loss = 'categorical_crossentropy',
              optimizer = optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics = ['accuracy'])

Found 744 images belonging to 3 classes.
Found 188 images belonging to 3 classes.




In [11]:
csv_logger = CSVLogger('food_recongition.log')
hist=model.fit_generator(
    train_generator,
    epochs=100,
    verbose=1,
    validation_data=validation_generator,
    callbacks=[csv_logger]
)

#save weights
model.save('food_recognition.h5')

Epoch 1/100
24/24 [==============================] - 25s 1s/step - loss: 0.9150 - acc: 0.5715 - val_loss: 0.7039 - val_acc: 0.7394
Epoch 2/100
24/24 [==============================] - 11s 468ms/step - loss: 0.7676 - acc: 0.6796 - val_loss: 0.7208 - val_acc: 0.6702
Epoch 3/100
24/24 [==============================] - 11s 470ms/step - loss: 0.6983 - acc: 0.6992 - val_loss: 0.5144 - val_acc: 0.8138
Epoch 4/100
24/24 [==============================] - 11s 465ms/step - loss: 0.6179 - acc: 0.7396 - val_loss: 0.4458 - val_acc: 0.8564
Epoch 5/100
24/24 [==============================] - 13s 533ms/step - loss: 0.5843 - acc: 0.7539 - val_loss: 0.4060 - val_acc: 0.8723
Epoch 6/100
24/24 [==============================] - 12s 481ms/step - loss: 0.5649 - acc: 0.7736 - val_loss: 0.3871 - val_acc: 0.8723
Epoch 7/100
24/24 [==============================] - 11s 474ms/step - loss: 0.5297 - acc: 0.7942 - val_loss: 0.3394 - val_acc: 0.8936
Epoch 8/100
24/24 [==============================] - 11s 470ms/st

In [0]:
model.save('drive/Colab Notebooks/food_recognition.h5')